In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('data/Hotel_details.csv')
df2 = pd.read_csv('data/Hotel_Room_attributes.csv')
df3 = pd.read_csv('data/hotel_price_min_max - Formula.csv')
df4 = pd.read_csv('data/hotels_RoomPrice.csv')

In [3]:
import pandas as pd

# Load the Hotel_details.csv dataset
hotel_details_path = 'data/Hotel_details.csv'
hotel_details_df = pd.read_csv(hotel_details_path)

# Load the url_image.txt file
url_image_path = 'url_image.txt'
with open(url_image_path, 'r') as file:
    url_images = file.read().splitlines()

# Assign url_image column to the dataset
# Ensure the url_images list is long enough; if not, cycle through the list
url_images_repeated = url_images * (len(hotel_details_df) // len(url_images)) + url_images[:len(hotel_details_df) % len(url_images)]
hotel_details_df['url_image'] = url_images_repeated

df1 = hotel_details_df

## Quick Data Check

In [4]:
df2.rename(columns = {'hotelcode':'hotelid'},inplace = True)
df_new = df2.merge(df1,on = 'hotelid')
df_new.isna().sum()
df_new.duplicated().sum()
df_new.drop(['zipcode'],axis = 1,inplace = True)
df_new.dropna(inplace = True)
df3.rename(columns  = {'hotelcode':'hotelid'},inplace = True)
pd.set_option('display.max_columns',30)
df6 = df_new.merge(df3,on = 'hotelid')
df6.duplicated().sum()

0

### Finally we will be using df as our main dataframe

In [5]:
df = df6
df.drop(['id_y','latitude','longitude','Source','curr'],axis = 1,inplace = True)
df.rename(columns = {'id_x':'id'},inplace = True)
df.duplicated().sum()
indices = df[df['id'].duplicated()].index
df.drop(index = indices,inplace = True)
df['id'].duplicated().sum()

0

## Data Cleaning

### 1. roomamenities

- Removing whitespaces is necessary for our the recommendation so that it treats the entire word as one single entity

In [6]:
def remove_whitespace(text):
   return  text.replace(' ','')

In [7]:
df['roomamenities'][0]

'Air conditioning: ;Alarm clock: ;Carpeting: ;Closet: ;Coffee/tea maker: ;Desk: ;DVD/CD player: ;Fan: ;Free Wi-Fi in all rooms!: ;Hair dryer: ;Heating: ;In-room safe box: ;Ironing facilities: ;Linens: ;Mini bar: ;Seating area: ;Toiletries: ;Towels: ;TV [flat screen]: ;'

In [8]:
df['roomamenities_tag'] = df['roomamenities'].apply(lambda x:x.replace(': ;',','))
df['roomamenities_tag'] = df['roomamenities_tag'].apply(remove_whitespace)
df['roomamenities_tag'] = df['roomamenities_tag'].apply(lambda x:x.split(','))

### 2. roomtype

In [9]:
df['roomtype_tag'] = df['roomtype'].apply(remove_whitespace)
df['roomtype_tag'] =  df['roomtype_tag'].apply(lambda x:[x])

### 3. ratedescription

- Remove room size because rooms with exact same size is not an important parameter for recommending hotel rooms

In [10]:
import re

def transform(text):
    return re.sub(r'Room size: \d+ m²/\d+ ft², ','',text)

df['ratedescription_tag'] = df['ratedescription'].apply(transform)
df['ratedescription_tag'] = df['ratedescription_tag'].apply(remove_whitespace)
df['ratedescription_tag'] = df['ratedescription_tag'].apply(lambda x:x.split(','))

### 4. address

In [11]:
df['address'].nunique()
df['address_tag'] = df['address'].apply(remove_whitespace)
df['address_tag'] = df['address_tag'].apply(lambda x:x.split())

### 5. city

In [12]:
df['city_tag'] = df['city'].apply(remove_whitespace)
df['city_tag'] = df['city_tag'].apply(lambda x:x.split())

### 6. country

In [13]:
df['country_tag'] = df['country'].apply(remove_whitespace)
df['country_tag'] = df['country_tag'].apply(lambda x:x.split())
df['country_tag'] 

0         [UnitedKingdom]
1         [UnitedKingdom]
2         [UnitedKingdom]
3         [UnitedKingdom]
4         [UnitedKingdom]
               ...       
175539           [France]
175540           [France]
175541           [France]
175542           [France]
175543           [France]
Name: country_tag, Length: 160438, dtype: object

### 7. propertytype

In [14]:
df['propertytype_tag'] = df['propertytype'].apply(remove_whitespace)
df['propertytype_tag'] = df['propertytype_tag'].apply(lambda x:x.split())

### Creating tags for each room of the hotels as a list of keywords which will serve as the basis for the recommendation system

In [15]:
df['tags'] = df['roomamenities_tag'] + df['roomtype_tag'] + df['ratedescription_tag'] + df['address_tag'] + df['city_tag'] + df['country_tag'] + df['propertytype_tag']
df['tags'] = df['tags'].apply(lambda x:" ".join(x))

### Filtering the important columns 

In [16]:
df

,id,hotelid,roomamenities,roomtype,ratedescription,hotelname,address,city,country,propertytype,starrating,url,url_image,min,max,Diff_Min,Diff_Max,Score,roomamenities_tag,roomtype_tag,ratedescription_tag,address_tag,city_tag,country_tag,propertytype_tag,tags
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed",The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,Hotels,4,https://www.agoda.com/en-gb/the-old-cider-hous...,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,82.36,107.70,0.084,0.418,0.25,"[Airconditioning, Alarmclock, Carpeting, Close...",[DoubleRoom],"[Shower, 1kingbed]",[25CastleStreet],[NetherStowey],[UnitedKingdom],[Hotels],Airconditioning Alarmclock Carpeting Closet Co...
1,50677498,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed",The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,Hotels,4,https://www.agoda.com/en-gb/the-old-cider-hous...,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,82.36,107.70,0.084,0.418,0.25,"[Airconditioning, Alarmclock, Carpeting, Close...",[DoubleRoom],"[Shower, 1kingbed]",[25CastleStreet],[NetherStowey],[UnitedKingdom],[Hotels],Airconditioning Alarmclock Carpeting Closet Co...
2,50677499,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Deluxe Double Room with Shower,"Room size: 17 m²/183 ft², Shower, 1 queen bed ...",The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,Hotels,4,https://www.agoda.com/en-gb/the-old-cider-hous...,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,82.36,107.70,0.084,0.418,0.25,"[Airconditioning, Alarmclock, Carpeting, Close...",[DeluxeDoubleRoomwithShower],"[Shower, 1queenbedor2singlebeds]",[25CastleStreet],[NetherStowey],[UnitedKingdom],[Hotels],Airconditioning Alarmclock Carpeting Closet Co...
3,50731279,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Superior Double Room,"Room size: 17 m²/183 ft², Shower, 1 double bed",The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,Hotels,4,https://www.agoda.com/en-gb/the-old-cider-hous...,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,82.36,107.70,0.084,0.418,0.25,"[Airconditioning, Alarmclock, Carpeting, Close...",[SuperiorDoubleRoom],"[Shower, 1doublebed]",[25CastleStreet],[NetherStowey],[UnitedKingdom],[Hotels],Airconditioning Alarmclock Carpeting Closet Co...
4,63480999,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Standard Double or Twin Room,"Room size: 13 m²/140 ft², Shower, 1 queen bed ...",The Old Cider House,25 Castle Street,Nether Stowey,United Kingdom,Hotels,4,https://www.agoda.com/en-gb/the-old-cider-hous...,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,82.36,107.70,0.084,0.418,0.25,"[Airconditioning, Alarmclock, Carpeting, Close...",[StandardDoubleorTwinRoom],"[Shower, 1queenbedor2singlebeds]",[25CastleStreet],[NetherStowey],[UnitedKingdom],[Hotels],Airconditioning Alarmclock Carpeting Closet Co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175539,88475965,6592926,Air conditioning: ;Dressing room: ;Free Wi-Fi ...,Standard Twin Room,"Room size: 16 m²/172 ft², 2 single beds",Kyriad Bordeaux Ouest - Merignac Aeroport,3 Rue Euler,Merignac,France,Hotels,3,https://www.agoda.com/en-gb/kyriad-bordeaux-ou...,20061391-326893946a4dfa8dbc7b7773b17503cb.png,768.54,1062.37,8.071,4.578,6.32,"[Airconditioning, Dressingroom, FreeWi-Fiinall...",[StandardTwinRoom],[2singlebeds],[3RueEuler],[Merignac],[France],[Hotels],Airconditioning Dressingroom FreeWi-Fiinallroo...
175540,88475966,6592926,Air conditioning: ;Dressing room: ;Free Wi-Fi ...,Standard Twin Room,"Room size: 16 m²/172 ft², 2 single beds",Kyriad Bordeaux Ouest - Merignac Aeroport,3 Rue Euler,Merignac,France,Hotels,3,https://www.agoda.com/en-gb/kyriad-bordeaux-ou...,20061391-326893946a4dfa8dbc7b7773b17503cb.png,768.54,1062.37,8.071,4.578,6.32,"[Airconditioning, Dressingroom, FreeWi

In [17]:
df_final = df[['hotelname','tags','roomtype','country','city','propertytype' , 'url_image' , 'roomamenities' , 'ratedescription', 'address' , 'min', 'max' , 'starrating']]
df_final.duplicated().sum()
df_final.drop_duplicates(inplace = True)
df_final.isna().sum()
df_final.reset_index(inplace = True)
df_final = df_final.head(30000)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26728\3877350181.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.drop_duplicates(inplace = True)


In [18]:
df_final

,index,hotelname,tags,roomtype,country,city,propertytype,url_image,roomamenities,ratedescription,address,min,max,starrating
0,0,The Old Cider House,Airconditioning Alarmclock Carpeting Closet Co...,Double Room,United Kingdom,Nether Stowey,Hotels,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 15 m²/161 ft², Shower, 1 king bed",25 Castle Street,82.36,107.70,4
1,2,The Old Cider House,Airconditioning Alarmclock Carpeting Closet Co...,Deluxe Double Room with Shower,United Kingdom,Nether Stowey,Hotels,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 17 m²/183 ft², Shower, 1 queen bed ...",25 Castle Street,82.36,107.70,4
2,3,The Old Cider House,Airconditioning Alarmclock Carpeting Closet Co...,Superior Double Room,United Kingdom,Nether Stowey,Hotels,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 17 m²/183 ft², Shower, 1 double bed",25 Castle Street,82.36,107.70,4
3,4,The Old Cider House,Airconditioning Alarmclock Carpeting Closet Co...,Standard Double or Twin Room,United Kingdom,Nether Stowey,Hotels,20011694-4c950f9b429acbf55d3751e720e401db.jpeg,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,"Room size: 13 m²/140 ft², Shower, 1 queen bed ...",25 Castle Street,82.36,107.70,4
4,10,Apollo Hotel London,Airconditioning Clothesrack Coffee/teamaker Da...,Standard Triple Room,United Kingdom,London,Hotels,20026625-39a5008d37c44a016fd9e40f1f1d4919.jpeg,Air conditioning: ;Clothes rack: ;Coffee/tea m...,"Room size: 20 m²/215 ft², Shower, 3 single beds",64-66 Queensborough Terrace,881.48,1055.92,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,126996,Padley Farm B&B,Airconditioning Alarmclock Closet Desk Freebot...,Deluxe King Room,United Kingdom,Bradford,Hotels,20074748-79a21380225e6964d3f52a2f71be3da4.jpeg,Air conditioning: ;Alarm clock: ;Closet: ;Desk...,"Room size: 14 m²/151 ft², Shower, 1 king bed",Dung worth Green,101.37,126.71,4
29996,126997,Padley Farm B&B,Airconditioning Alarmclock Closet Desk Freebot...,Luxury Triple Room,United Kingdom,Bradford,Hotels,20074748-79a21380225e6964d3f52a2f71be3da4.jpeg,Air conditioning: ;Alarm clock: ;Closet: ;Desk...,"Room size: 20 m²/215 ft², Balcony/terrace, Sho...",Dung worth Green,101.37,126.71,4
29997,126998,Padley Farm B&B,Airconditioning Alarmclock Closet Desk Freebot...,Family Room with Shower,United Kingdom,Bradford,Hotels,20074748-79a21380225e6964d3f52a2f71be3da4.jpeg,Air conditioning: ;Alarm clock: ;Closet: ;Desk...,"Room size: 24 m²/258 ft², Balcony/terrace, Sho...",Dung worth Green,101.37,126.71,4
29998,126999,Cooneys Hotel,Airconditioning Carpeting Closet Clothesrack D...,Double Room,Ireland,Ballymahon,Hotels,67845160-41c37fc5eeef55cfc2af5f323ea110e1.jpeg,Air conditioning: ;Carpeting: ;Closet: ;Clothe...,"Room size: 30 m²/323 ft², Shower and bathtub, ...",Main Street,136.31,136.31,3


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
import pickle
pickle.dump(df_final,open('data.pkl','wb'))
df_final.to_csv("data.csv" , index = False)

## Recommender Function

In [22]:
import pickle

def recommend(type,country,city,property):
    
    ## filtering the dataframe to the requirements of the user
    filtered_df = df_final[
        (df_final['roomtype'] == type) &
        (df_final['country'] == country) &
        (df_final['city'] == city) &
        (df_final['propertytype'] == property)
    ]

    ## creating a temporary dataframe so that we recommend hotels only from the speicified country and city rather than using the entire df which
    ## could lead to results from different cities and countries
    temp = df_final[
        (df_final['country'] == country) &
        (df_final['city'] == city) 
    ]

    ## indices of the hotels of user requirements
    idx1 = filtered_df['index'].tolist()

    ## reseting index because we need to search for the indices in idx1 in temp
    temp.reset_index(inplace = True)

    ## extracting the index(of the dataframe) of the specified hotels 
    idx2 = temp[temp['index'].isin(idx1)].index.tolist()
    
    ## creating similarity matrix
    vector = tfidf.fit_transform(temp['tags']).toarray()
    similarity = cosine_similarity(vector)

    # pickle.dump(similarity,open('similarity.pkl','wb'))
    # pickle.dump(df_final,open('data.pkl','wb'))
    recommended_movie_names = []
    ## traverse each user specified hotel and extract top recommend hotels for each specified hotel
    for i in idx2:
        similar_hotels = sorted(list(enumerate(similarity[i])),key = lambda x:x[1],reverse = True)[0:7]
        for hotel in similar_hotels:
            recommended_movie_names.append(temp.loc[hotel[0]])
            
    return recommended_movie_names

In [23]:
a = recommend('Comfort Single Room','Germany','Munich','Hotels')

In [24]:
a[0]["max"]

120.84

In [25]:
recommend('Superior Double Room','France','Paris','Hotels')

[level_0                                                           16
 index                                                             54
 hotelname                                       Timhotel Tour Eiffel
 tags               Airconditioning Alarmclock Bathrobes Blackoutc...
 roomtype                                        Superior Double Room
 country                                                       France
 city                                                           Paris
 propertytype                                                  Hotels
 url_image             67845160-41c37fc5eeef55cfc2af5f323ea110e1.jpeg
 roomamenities      Air conditioning: ;Alarm clock: ;Bathrobes: ;B...
 ratedescription    Room size: 14 m²/151 ft², City view, Non-smoki...
 address                                                  11 rue Juge
 min                                                           104.48
 max                                                           236.09
 starrating         